* URL shortening is used to create shorter aliases for long URLs.
* URL shortening is used for optimizing links across devices, tracking individual links to analyze audience and campaign performance, and hiding affiliated original URLs.

# Requirements

#### Functional
- Given URL, generate unique and shorter link.
- When user click on short link, redirect them to original link
- Custom short link
- Link expire after default TTL. User can also specify TTL

#### Non-functional
- Highly available system
- URL refirection latency should be minimal and in real time
- Shortened link should not be guessable

#### Extended
- Analystics. how many time link opened?
- REST API for our service.

# Capacity and Estimation 

* Read heavy system
* 100:1 ratio for read and write
* Assume that 500M new URL shortening per month, with 100:1 ration 500 * 100 M = 50 B redirections.
* Shortening request per second
    - 500 M / (30 * 24 * 3600) = 200 URL / second
* Read request per second
    - 50 B / (30 * 24 * 3600) = 20K URL / second
* Storage need
    - Assume that each URL uses 500 bytes, we save data for 5 years
    - 500M * 5 * 12 = 30 billion
    - 30 * 500 = 15 TB
* Bandwidth need
    - For write request 200 * 500 bytes = 100 KB/s
    - for read 20K * 500 bytes = 10 MB/s
* Memory need
    - To cache some hot URLs.
    - 80-20 rule, 20% URLs generates 80% traffic
    - 20K request per second => 1.7 billion request per day
    - To cache 20% of these request 0.2  * 1.7 billion * 500 bytes = 170GB (Lot of the request are duplicates, memoryneed will be lesser)

# System API
* This will show what we need from the system
* Create URL

```
createURL(api_dev_key (string), original_url (string), custom_alias=None (string), user_name=None (string), expire_date=None (string))
```
* api_dev_key can be used to control allowed quota of user
* Returns string

* Delete URL

```
deleteURL(api_dev_key, url_key)
```
* url_key is shortend url
* Success message is returned.

* A malicious user can put us out of business by consuming all URL keys in the current design. To prevent abuse, we can limit users via their api_dev_key. Each api_dev_key can be limited to a certain number of URL creations and redirections per some time period (which may be set to a different duration per developer key).

# Database Design
* We need to store billions of records
* Each object is small
* No relationship between records (we need to store which user created which URLs)
* read heavy service
![](images/tiny_url_database.SVG)

* billions of rows and no relationship between various objects-NOSQL key-value store like Cassandra, DynamoDB, Riak is better choice. NoSql is easier to scale
* Key will be the tiny url and value is longer URL.

# Algorithms and System Design

* How to generate unique key for given URL. We want to generate key `http://tinyurl.com/jlg8zpc` like `jlg8zpc`
    - We can use encoding base36 ([a-z, 0-9]) or base64 ([a-z, A-Z, 0-9])
    - What should be the length of key 6, 8, 10
    - Using base64 encoding 64^6 (A-Z, a-Z, 0-9, +, /) = 68.7 billion and 64^8 = 281 trillion
    - base62 also can be used.
    - 68.7B is enough for us.
* If we use the MD5 algorithm as our hash function, it’ll produce a 128-bit hash value. After base64 encoding, we’ll get a string having more than 21 characters (since each base64 character encodes 6 bits of the hash value). Since we only have space for 8 characters per short key, how will we choose our key then? We can take the first 6 (or 8) letters for the key. This could result in key duplication, to resolve that, we can choose some other characters out of the encoding string or swap some characters.
    * If multiple users enter the same URL, they can get the same shortened URL, which is not acceptable.
    * What if parts of the URL are URL-encoded? 
* We can append an increasing sequence number to each input URL to make it unique, and then generate a hash of it. We don’t need to store this sequence number in the databases, though. Possible problems with this approach could be an ever-increasing sequence number. Can it overflow? Appending an increasing sequence number will also impact the performance of the service.
* Another solution could be to append user id (which should be unique) to the input URL. However, if the user has not signed in, we would have to ask the user to choose a uniqueness key. Even after this, if we have a conflict, we have to keep generating a key until we get a unique one.
![](images/Tiny_url1.PNG)

### Generating keys offline
* We can hav standalone key generation service, which can create a keys beforehand and store it in db. To short new URL use already generated key. No worry for encoding the URL, duplication or collision.
* Concurrency:
    - As soon as key is used mark it as used in db. What if 2 or  more server read same key concurrently.
    - Servers can use KGS to read/mark keys in the database. KGS can use two tables to store keys: one for keys that are not used yet, and one for all the used keys. As soon as KGS gives keys to one of the servers, it can move them to the used keys table. KGS can always keep some keys in memory so that it can quickly provide them whenever a server needs them.
    - For simplicity, as soon as KGS loads some keys in memory, it can move them to the used keys table. This ensures each server gets unique keys. If KGS dies before assigning all the loaded keys to some server, we will be wasting those keys–which is acceptable, given the huge number of keys we have.
    - KGS also has to make sure not to give the same key to multiple servers. For that, it must synchronize (or get a lock on) the data structure holding the keys before removing keys from it and giving them to a server
    - What would be the key-DB size? With base64 encoding, we can generate 68.7B unique six letters keys. If we need one byte to store one alpha-numeric character, we can store all these keys in: 6 * 68.7B = 412GB
    - Being single point of failure we canuse standby replica of KGS
    - Each app server can cache some keys from key-db. Which can speed up thing. But if the app server dies, we will end up losing those keys.
    - We can look up the key in our database or key-value store to get the full URL. If it’s present, issue an “HTTP 302 Redirect” status back to the browser, passing the stored URL in the “Location” field of the request. If that key is not present in our system, issue an “HTTP 404 Not Found” status or redirect the user back to the homepage.
    -  Users can pick any ‘key’ they like, but providing a custom alias is not mandatory. However, it is reasonable (and often desirable) to impose a size limit on a custom alias to ensure we have a consistent URL database. 

# Data Partitioning and Replication
* To scale out our DB, we need to partition it so that it can store information about billions of URLs. We need to come up with a partitioning scheme that would divide and store our data into different DB servers.
*  Range Based Partitioning: We can store URLs in separate partitions based on the first letter of the hash key. Hence we save all the URLs starting with letter ‘A’ in one partition, save those that start with letter ‘B’ in another partition and so on. This approach is called range-based partitioning. We can even combine certain less frequently occurring letters into one database partition. We should come up with a static partitioning scheme so that we can always store/find a URL in a predictable manner.

* The main problem with this approach is that it can lead to unbalanced DB servers. For example, we decide to put all URLs starting with letter ‘E’ into a DB partition, but later we realize that we have too many URLs that start with the letter ‘E’.

* Hash-Based Partitioning: In this scheme, we take a hash of the object we are storing. We then calculate which partition to use based upon the hash. In our case, we can take the hash of the ‘key’ or the short link to determine the partition in which we store the data object. This approach can still lead to overloaded partitions, which can be solved by using Consistent Hashing.

# Cache
* Memcached
* How much cache memory
    - We can start with 20% of daily traffic and, based on clients’ usage pattern, we can adjust how many cache servers we need. As estimated above, we need 170GB memory to cache 20% of daily traffic. Since a modern-day server can have 256GB memory, we can easily fit all the cache into one machine. Alternatively, we can use a couple of smaller servers to store all these hot URLs.
- To further increase the efficiency, we can replicate our caching servers to distribute the load between them.
- Least Recently Used (LRU) can be a reasonable policy for cache eviction
-  Whenever there is a cache miss, our servers would be hitting a backend database. Whenever this happens, we can update the cache and pass the new entry to all the cache replicas. Each replica can update its cache by adding the new entry. If a replica already has that entry, it can simply ignore it.
- If URL not found in db or cache or expired return HTTP error 401.

# Load Balancer

* Between Clients and Application servers
* Between Application Servers and database servers
* Between Application Servers and Cache servers
*  if a server is dead, LB will take it out of the rotation and will stop sending any traffic to it. A problem with Round Robin LB is that we don’t take the server load into consideration. If a server is overloaded or slow, the LB will not stop sending new requests to that server. To handle this, a more intelligent LB solution can be placed that periodically queries the backend server about its load and adjusts traffic based on that.

# Purging or DB cleanup

* If we chose to actively search for expired links to remove them, it would put a lot of pressure on our database. Instead, we can slowly remove expired links and do a lazy cleanup. Our service will make sure that only expired links will be deleted, although some expired links can live longer but will never be returned to users.
* Whenever a user tries to access an expired link, we can delete the link and return an error to the user.
* A separate Cleanup service can run periodically to remove expired links from our storage and cache. This service should be very lightweight and can be scheduled to run only when the user traffic is expected to be low.
* We can have a default expiration time for each link (e.g., two years).
* After removing an expired link, we can put the key back in the key-DB to be reused.
* Should we remove links that haven’t been visited in some length of time, say six months? This could be tricky. Since storage is getting cheap, we can decide to keep links forever.

![](images/Tiny_url2.PNG)

# Telemetry

* country of the visitor, date and time of access, web page that refers the click, browser, or platform from where the page was accessed.

# Security and Permissions
* Can users create private URLs or allow a particular set of users to access a URL?
* We can store the permission level (public/private) with each URL in the database. We can also create a separate table to store UserIDs that have permission to see a specific URL. If a user does not have permission and tries to access a URL, we can send an error (HTTP 401) back. Given that we are storing our data in a NoSQL wide-column database like Cassandra, the key for the table storing permissions would be the ‘Hash’ (or the KGS generated ‘key’). The columns will store the UserIDs of those users that have the permission to see the URL.

* We need bijective function for key generation
    - Each x associated only one y and vice versa.
![](images/Tiny_url3.PNG)

![](images/tiny4.PNG)